<a href="https://colab.research.google.com/github/Mallesh06/Smart-Rain-Detection-Automated-Irrigation/blob/main/rain_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers torch torchvision datasets pillow

In [2]:
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer
from datasets import load_dataset
from torchvision import transforms
from PIL import Image
import torch

In [3]:
model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name, num_labels=3) # Changed num_labels to 3

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/Colab Notebooks/dataset")

# Verify
print(dataset)

Resolving data files:   0%|          | 0/468 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/298 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 468
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 298
    })
})


In [5]:
normalize = transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize,
])

In [6]:
training_args = TrainingArguments(
    output_dir="./vit-finetuned-rain",
    per_device_train_batch_size=8,
    eval_strategy="steps",
    num_train_epochs=3,  # You can increase to 5–10 if you have GPU time
    save_steps=250,
    save_total_limit=2,
    logging_steps=10,
    learning_rate=2e-5,
    push_to_hub=False,
    report_to="none",  # Disable Weights & Biases logging
)

In [7]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [8]:
def preprocess_image(examples):
    # Apply the defined transformations to the 'image' column
    examples["pixel_values"] = [train_transforms(image.convert("RGB")) for image in examples["image"]]
    # Return only the processed pixel values and the labels
    return {"pixel_values": examples["pixel_values"], "label": examples["label"]}

# Apply the preprocessing function to the dataset
processed_dataset = dataset.map(preprocess_image, batched=True)

# Remove the original 'image' column as it's no longer needed
processed_dataset = processed_dataset.remove_columns(["image"])

# Verify the structure of the processed dataset
print(processed_dataset)

Map:   0%|          | 0/468 [00:00<?, ? examples/s]

Map:   0%|          | 0/298 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'pixel_values'],
        num_rows: 468
    })
    test: Dataset({
        features: ['label', 'pixel_values'],
        num_rows: 298
    })
})
